### Лекция 9. Алгоритмы и лямбды

https://en.cppreference.com/w/cpp/algorithm

https://en.cppreference.com/w/cpp/numeric

https://en.cppreference.com/w/cpp/language/lambda

<br />

##### Примеры и философия алгоритмов

`accumulate` для знакомства

In [ ]:
int sum_98(const std::vector<int>& v)
{
    int rv = 0;
    for (std::vector<int>::const_iterator it = v.begin(), e = v.end(); it != e; ++it)'
        rv += *it;
    return rv;
}

int sum_range_for_11(const std::vector<int>& v)
{
    int rv = 0;
    for (int x : v)
        rv += x;
    return rv;
}

int sum_stl_accumulate(const std::vector<int>& v)
{
    // # https://en.cppreference.com/w/cpp/algorithm/accumulate
    return std::accumulate(v.begin(), v.end(), 0);
    
    // # как это читать:
    // # std::accumulate(  // просуммировать элементы из последовательности (std::plus - дефолтная операция у accumulate)
    // #      v.begin(),   // итератор на первый элемент в последовательности
    // #      v.end(),     // итератор на следующий за последним элементом в последовательности
    // #      0);          // начальное значение суммы
}

int sum_stl_reduce(const std::vector<int>& v)
{
    // # https://en.cppreference.com/w/cpp/algorithm/reduce (C++17)
    return std::reduce(v.begin(), v.end());
}

`count` - ещё пример

In [ ]:
size_t count_range_for_11(const std::vector<int>& v, int val)
{
    size_t rv = 0;
    for (auto x : v)
        if (x == val)
            ++rv;
    return rv;
}

size_t count_stl(const std::vector<int>& v, int val)
{
    return std::count(v.begin(), v.end(), val);
}

`find`

In [ ]:
bool contains_range_for_11(const std::vector<int>& v, int val)
{
    for (int x : v)
        if (x == val)
            return true;
    return false;
}

bool contains_stl(const std::vector<int>& v, int val)
{
    return std::find(v.begin(), v.end(), val) != v.end();
}

На примере `find` обсудить вопрос что лучше: наивная реализация или переиспользование алгоритма:
* какой код чище?
* вопрос уровня абстракции
* оптимизации внутри алгоритмов
* проблема читабельности алгоритмов:
    * сколько служебных конструкций и повторений содержит элементарный код проверки, что `v` содержит `val`?
    
    ```
    std::find(
        current_students_in_the_class.begin(),
        current_students_in_the_class.end(),
        "those little boy Einstein")
        != current_students_in_the_class.end();
    ```
    
* ranges to the rescue! (но не совсем и с кучей оговорок)

<br />

##### Алгоритмы работают с итераторами!

Класс итератора обладает некоторой гибкостью, что позволяет делать весьма заковыристые конструкции

Пример: разные варианты сортировки через один алгоритм

In [ ]:
// # отсортируем вектор по возрастанию
void mysort_ascending(std::vector<int>& v)
{
    std::sort(v.begin(), v.end());
}


// # отсортируем вектор по убыванию
void mysort_descending(std::vector<int>& v)
{
    std::sort(v.rbegin(), v.rend());
}

Пример: вывод на консоль через алгоритм (подробно объяснить пример)

In [ ]:
// # copy - копирует последовательность
// #        https://en.cppreference.com/w/cpp/algorithm/copy
void copy_example()
{
    std::vector<int> v = {1, 2, 3};
    std::list<int> l = {4, 5, 6};
    
    std::copy(v.begin(), v.end(), l.begin());
}


// # copy - копирует последовательность в OutputIt,
// #        но ведь можно и подшаманить OutputIt
void print_sequence()
{
    std::vector<int> v = {1, 2, 3, 4, 5};
    
    std::copy(v.begin(), v.end(),
              std::ostream_iterator<int>(std::cout, " "));
}

Пример: чтение (подробно объяснить пример)

In [ ]:
std::vector<int> read_ints()
{
    return {
        std::istream_iterator<int>(std::cin),
        std::istream_iterator<int>()       
    };
}


int sum_ints()
{
    std::istringstream iss("1 2 3 4 5");
    
    return std::reduce(
        std::istream_iterator<int>(iss),
        std::istream_iterator<int>());
}

Шутки ради можно показать в какой дикий ассемблер разворачивается однострочник `read_ints`

<br />

И в то же время гибкости класса итераторов не хватает, чтобы простые конструкции реализовывались без лишних раздумий:

In [ ]:
void copy_example_bug()
{
    std::vector<int> a = {1, 2, 3, 4, 5};
    std::vector<int> b;

    std::copy(a.begin(), a.end(), b.begin());
}


void copy_example_fix1()
{
    std::vector<int> a = {1, 2, 3, 4, 5};
    std::vector<int> b;

    b.resize(a.size(), 0);
    std::copy(a.begin(), a.end(), b.begin());
}


void copy_example_fix2()
{
    std::vector<int> a = {1, 2, 3, 4, 5};
    std::vector<int> b;

    std::copy(a.begin(), a.end(), std::back_inserter(b));
    // # back_inserter - специальный итератор, который в operator = вызывает b.push_back(...)
}

<br />

In [ ]:
void replace_copy_bug()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    std::vector<int> b;

    std::replace_copy(a.begin(), a.end(), b.begin(), 2, 7);
}


void replace_copy_fix()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    std::vector<int> b;

    std::replace_copy(a.begin(), a.end(), std::back_inserter(b), 2, 7);
}


void replace_copy_better()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    std::vector<int> b;

    b.reserve(a.size());
    std::replace_copy(a.begin(), a.end(), std::back_inserter(b), 2, 7); // # O(n)
}

<br />

##### `std::remove` и `std::unique` - где чаще всего ошибаются

In [ ]:
void remove_usage_bug()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    std::remove(a.begin(), a.end(), 2);
}


void remove_usage_fix()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    auto new_end = std::remove(a.begin(), a.end(), 2);
    a.resize(std::distance(a.begin(), new_end));
}


void remove_usage_list()
{
    std::list<int> l = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    l.remove(2); // # list is an exception!
}

In [ ]:
void unique_usage_bug()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    std::sort(a.begin(), a.end());  // # unique removes only adjacent uniques
    std::unique(a.begin(), a.end());
}


void unique_usage_fix()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    std::sort(a.begin(), a.end()); // # unique removes only adjacent uniques
    auto new_end = std::unique(a.begin(), a.end());
    a.resize(std::distance(a.begin(), new_end));
}


void unique_usage_list()
{
    std::list<int> l = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    l.sort();  // # unique removes only adjacent uniques
    l.unique();
}

<br />

Обратить внимание на состояние контейнера после вызова remove:

In [ ]:
void remove_result_content()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    auto new_end = std::remove(a.begin(), a.end(), 2);
    
    for (int x : a)
        std::cout << x << " "; // # ???
}

<br />

##### Алгоритмы и лямбды

`find` по условию:

In [ ]:
bool is_underage(const Person& p) {
    return p.age < 18;
}

std::vector<Person> people = { ... };

auto it = std::find_if(
    people.begin(),
    people.end(),
    is_underage);

if (it != people.end())
    std::cout << it->age << std::endl;

In [ ]:
std::vector<Person> people = { ... };

auto it = std::find_if(
    people.begin(),
    people.end(),
    [](const Person& p){ return p.age < 18; });
    
if (it != people.end())
    std::cout << it->age << std::endl;

<br />

`count_if` - подсчёт числа элементов

In [ ]:
std::vector<int> v = { ... };

const int fives_count = std::count(v.begin(), v.eng(), 5);

In [ ]:
std::vector<Person> people = { ... };
        
const int underage_count = std::count(people.begin(), people.end(),
                                      [](const Person& p){ return p.age < 18; });

<br />

`all_of`, `any_of`, `none_of`

In [ ]:
auto is_underage = [](const Person& p){ return p.age < 18; };

bool all_young = std::all_of(people.begin(), people.end(), is_underage);
bool any_young = std::any_of(people.begin(), people.end(), is_underage);
bool all_old   = std::none_of(people.begin(), people.end(), is_underage);

<br />

`transform`

In [ ]:
void toupper_inplace(std::string& s)
{
    std::transform(
        s.begin(), s.end(),  // # input range
        s.begin(),           // # output
        [](unsigned char c) -> unsigned char { return std::toupper(c); });  // # function
}


void print_toupper(std::string& s)
{
    std::tranform(
        ...
        // # Упражнение
    );
}


// # объяснить этот пример подробнее
int sum_of_squarries(const std::vector<int>& v)
{
    return std::transform_reduce(
        v.begin(), v.end(),                     // # последовательность
        0,                                      // # начальный элемент в reduce-шаге (как в accumulate)
        [](int sum, int x) { return sum + x; }, // # операция reduce
        [](int x){ return x * x; });            // # операция transform
}

<br />

binary search:

In [ ]:
std::vector<int> v = { 1, 1, 2, 3, 4, 4, 4, 5, 5, 6 };
        
auto lower = std::lower_bound(v.begin(), v.end(), 4);
auto upper = std::upper_bound(v.begin(), v.end(), 4);
        
std::copy(lower, upper,
          std::ostream_iterator<int>(std::cout, " "));
// # output: 4 4 4

In [ ]:
if (std::binary_search(v.begin(), v.end(), 5))
    std::cout << "found";
else
    std::cout << "not found";

<br />

set operations:

**Замечание**: специфика set operations, что они работают только с отсортированными последовательностями

In [ ]:
std::vector<int> v1 = {...};
std::sort(v1.begin(), v1.end());
        
std::vector<int> v2 = {...};
std::sort(v2.begin(), v2.end());
        

// # check for subset:
if (std::includes(v1.begin(), v1.end(),
                  v2.begin(), v2.end())) {
    std::cout << "v2 is subset of v1";
}


// # build union
std::vector<int> v3 = {...};
std::set_union(v1.begin(), v1.end(),  // # отсортированная последовательность 1
               v2.begin(), v2.end(),  // # отсортированная последовательность 2
               std::back_inserter(v3));  // # куда складывать результат
// # что здесь можно было бы добавить?

<br />

min/max

In [ ]:
std::vector<int> v = { ... };

auto it_min = std::min_element(v.begin(), v.end());
auto it_max = std::max_element(v.begin(), v.end());
// # почему возвращается итератор, а не сразу ссылка на минимальный элемент?

In [ ]:
std::vector<Person> people = { ... };
        
auto it_min = std::min_element(people.begin(), people.end(),
                               [](const Person& l, const Person& r) {
                                   return l.name < r.name;
                               });  // # поиск первого по алфавиту

auto it_max = std::max_element(people.begin(), people.end(),
                               [](const Person& l, const Person& r) {
                                   return l.age < r.age;
                               });  // # поиск самого старшего

<br />

etc. etc. etc.

<br />

##### Execution policies

С С++17 можно запускать некоторые алгоритмы в параллельное исполнение одним дополнительным аргументом:

In [ ]:
std::reduce(std::execution::par, v.begin(), v.end());

* `std::execution::seq`
    - последовательно на одном потоке
    - порядок обработки элементов соответствует их порядку в последовательности
* `std::execution::par`
    - STL вправе (но не обязана) выполнить алгоритм параллельно на нескольких потоках
    - порядок обработки элементов на одном потоке соответствует их порядку в последовательности
* `std::execution::unseq`
    - последовательно на одном потоке
    - порядок обработки элементов может не соответствовать их порядку в последовательности (например, рарешается применять векторизацию)
* `std::execution::par_unseq`
    - STL вправе (но не обязана) выполнить алгоритм параллельно на нескольких потоках
    - порядок обработки элементов произвольный

Заметки про производительность ради которой всё это затевалось:
https://www.bfilipek.com/2018/11/parallel-alg-perf.html

Спойлер: не всё так однозначно

<br />

За рамками рассказа:
* boost algorithms
* bit manipulations
* math operations
* другие реализации алгоритмов (EASTL)